<a href="https://colab.research.google.com/github/AnnCherk/LabsMOMO/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа 2: Использование численных методов в задачах оптимизации
_Команда 9_ <br>
_Куимов, Оплетаев, Подольская, Черкасская_
### 1. Поиск минимума функции:

\begin{align}
f(x)= \frac {1}{2} [ (x_ {1})^ {2} + \sum _ {i=1}^ {2} (x_ {i}-x_ {i+1})^ {2} + (x_ {3})^ {2} ]- x_ {1}
\end{align}


##### Представление применимости алгоритмов BFGS и L-BFGS к данной оптимизационной задаче

Чтобы алгоритмы BFGS и L-BFGS могли быть использованы в данной задаче оптимизации необходимо, чтобы функция была дваждый дифференцирума и выпукла. Докажем это.

Первые производные:
\begin{align}
        \frac{df}{dx_1}=2x_1-x_2-1, \frac{df}{dx_2}=-x_1+2x_2-x_3, \frac{df}{dx_3}=-x_2+2x_3
    \end{align}

Вторые производные:
\begin{align}
        \frac{df}{dx_1^2}=2, \frac{df}{dx_2^2}=2, \frac{df}{dx_3^2}=2
    \end{align}

Смешанные производные:
\begin{align}
        \frac{df}{dx_1 dx_2}=-1 ,  \frac{df}{dx_1 dx_3 }=0 ,  \frac{df}{dx_2 dx_1}=-1, \frac{df}{dx_2 dx_3}=-1 , \frac{df}{dx_3 dx_1}=0 ,  \frac{df}{dx_3 dx_2}=-1
    \end{align}

Целвая функция дважды дифференцируема.

Далее проверим функцию на выпуклость. Для этого построим матрицу вторых производных Гессе – H.

\begin{align}
        H = \begin{pmatrix}
        \frac{df}{dx_1^2} & \frac{df}{dx_1 dx_2} &  \frac{df}{dx_1 dx_3}\\
        \frac{df}{dx_2 dx_1} & \frac{df}{dx_2^2} &  \frac{df}{dx_2 dx_3} \\
        \frac{df}{dx_3 dx_1} & \frac{df}{dx_3 dx_2} &  \frac{df}{dx_3^2}
        \end{pmatrix}=\begin{pmatrix}
        2 & -1 &  0\\
        -1 & 2 &  -1 \\
        0 & -1 &  2
        \end{pmatrix}.
    \end{align}

Если матрица Гессе положительно определенная или положутельно полуопределенная, то функция – выпуклая. Проверим собственные числа матрицы H.

In [ ]:
import numpy as np

In [ ]:
H=np.array([[2, -1, 0],[-1, 2, -1],[0, -1, 2]])
eigenvalues = np.linalg.eigvals(H)

print("Собственные числа матрицы H:")
print(eigenvalues)

Собственные числа матрицы H:
[3.41421356 2.         0.58578644]


Все собственные значения матрицы H положительны, то матрица Гессе положительно определена, и функция является строго выпуклой.

Таким образом, алгоритмы BFGS и L-BFGS применимы к данной задаче.


##### Реализация алгоритмов BFGS и L-BFGS

In [ ]:
# Представление функции
def f(x):
  x1=x[0]
  x2=x[1]
  x3=x[2]
  return 0.5*(x1**2+(x1-x2)**2+(x2-x3)**2+x3**2)-x1

In [ ]:
# Аналитическое представление градиента
def grad_f(x):
  x1=x[0]
  x2=x[1]
  x3=x[2]
  return np.array([2*x1-x2-1, -x1+2*x2-x3, -x2+2*x3])

In [ ]:
# Численное задание градиента
def num_grad_f(x, f=f, h=1e-6):
    grad=np.zeros(len(x))

    for i in range(len(x)):
        x_plus_h=x.copy()
        x_minus_h=x.copy()

        x_plus_h[i]+=h
        x_minus_h[i]-=h

        grad[i]=(f(x_plus_h)-f(x_minus_h))/(2 * h)

    return grad

In [ ]:
#Функции линейного поиска величины шага
def zoom(f, grad, x, p, a_low=0, a_high=1e5, c1=0.1, c2=0.9, max_iter = 100):
    i = 0

    while i<max_iter:
        alpha_j = (a_low + a_high) / 2
        beta_j = f(x + alpha_j*p)
        gamma_j = grad(x + alpha_j*p).dot(p)

        beta_low = f(x + a_low*p)

        if (beta_j > f(x) + c1*alpha_j*grad(x).dot(p)) or (beta_j >= beta_low):
            a_high = alpha_j
        else:
            if abs(gamma_j) <= -c2*grad(x).dot(p):
                return alpha_j

            if gamma_j*(a_high - a_low) >= 0:
                a_high = a_low

            a_low = alpha_j

        i+=1

    return a_low


def line_search(f, grad, x, p, c1=0.1, c2=0.9, a_max=1, max_iter=100):
    alpha=a_max
    i=0

    phi0 = f(x)
    beta0 = grad(x).dot(p)

    while i<max_iter:
        phi = f(x+alpha*p)

        if (phi>phi0+c1*alpha*beta0) or ((phi >= phi0) and (i>0)):
            return zoom(f, grad, x, p, alpha, a_max, c1, c2)

        beta=grad(x+alpha*p).dot(p)

        if abs(beta) <= -c2*beta0:
            return alpha

        if beta >= 0:
            return zoom(f, grad, x, p, alpha, a_max, c1, c2)

        alpha=alpha*2
        i+=1

    return alpha

In [ ]:
# Функции оптимизации
def bfgs_optimizer(f, grad, x0, H_init, max_iter=100, tol=1e-6):
    n = len(x0)
    x = x0.copy()
    H = H_init(x)  # Инициализация аппроксимации обратной гессианы
    g = grad(x)  # вычисление градиента в начальной точке

    for _ in range(max_iter):
        p=-np.dot(H, g)  # вычисление направления спуска

        # Поиск величины шага
        alpha = line_search(f, grad, x, p)

        x_new = x + alpha * p  # Новая точка

        g_new = grad(x_new)
        y=g_new - g
        s=x_new - x

        # Обновление приближения обратной гессианы с использованием формулы BFGS
        A = np.eye(n) - np.outer(s, y)/np.dot(y, s)
        H = np.dot(A, np.dot(H, np.transpose(A)))+np.outer(s, s)/np.dot(y, s)

        # Проверка условия сходимости
        if np.linalg.norm(g_new) < tol:
            break

        x = x_new
        g = g_new

    return x


def lbfgs_optimizer(f, grad, x0, H_init, m=100, max_iter=1000, eps=1e-6):
    # Инициализация
    x = x0.copy()
    n = len(x)
    s = np.zeros((m, n))
    y = np.zeros((m, n))
    phi = np.zeros(m)
    alpha = np.zeros(m)

    # Инициализация матрицы Гессе
    H = H_init(x)

    for k in range(max_iter):
        # Вычисление градиента
        g = grad(x)

        # Проверка условия остановки
        if k > 0 and abs(f(x) - f(x_prev)) < eps:
            break

        if k < m:
            p = -np.dot(H, g) # Определения направления шага
        else:
            q = g.copy()
            for i in range(m-1, -1, -1):
                alpha[i] = phi[i] * np.dot(s[i], q)
                q -= alpha[i] * y[i]
            p = q

        # Адаптивный шаг поиска
        alpha_k = 1.0
        x_new = x + alpha_k * p
        while f(x_new) > f(x) + 1e-4 * alpha_k * np.dot(g, p):
            alpha_k *= 0.5
            x_new = x + alpha_k * p

        # Обновление памяти
        s[k % m] = x_new - x
        y[k % m] = grad(x_new) - g

        if np.dot(y[k % m], s[k % m]) == 0:
            phi[k % m] = 0
        else:
            phi[k % m] = 1.0 / np.dot(y[k % m], s[k % m])

        x_prev = x.copy()
        x = x_new

        # Обновление матрицы Гессе
        if k >= m:
            r = np.dot(H, y[(k-1) % m])
            gamma = np.dot(s[(k-1) % m], y[(k-1) % m]) / np.dot(y[(k-1) % m], y[(k-1) % m])
            H += np.outer(s[(k-1) % m], s[(k-1) % m]) / np.dot(y[(k-1) % m], s[(k-1) % m]) - np.outer(r, r) / np.dot(y[(k-1) % m], r) + gamma * np.outer(r, r)

    return x

##### Приближение матрицы Гессе

In [ ]:
#Инициализация приближения матрицы Гессе через единичную матрицу
def H_ones(x):
  H = np.eye(len(x))
  return H


#Инициализация приближения матрицы Гессе через вычисленные на основе конечных разностей частные производные целевой функции
def H_approx_initial(x, f=f, h=1e-6):
  H = np.zeros((len(x), len(x)))

  for i in range(len(x)):
      for j in range(len(x)):
          e_i = np.zeros(len(x))
          e_i[i] = 1
          e_j = np.zeros(len(x))
          e_j[j] = 1

          H[i][j] = (f(x + h * e_i + h * e_j) - f(x + h * e_i) - f(x + h * e_j) + f(x)) / (h**2)

  return H

##### Сравнение алгоритмов BFGS и L-BFGS

In [ ]:
# Начальное приближение
x0 = [-1, 0, 110]

In [ ]:
import time

#Сравнение алгоритмов при применении аналитического и численного градиента при инициализации Гессиана единичной матрицей
start_time = time.time()
result_bfgs1=[round(num, 5) for num in bfgs_optimizer(f, grad_f, x0, H_ones)]
end_time = time.time()
print("1. Оптимальная точка(BFGS с аналитическим градиентом):", result_bfgs1)
print("Значение целевой функции в оптимальной точке:", round(f(result_bfgs1), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")


start_time = time.time()
result_bfgs2=[round(num, 5) for num in bfgs_optimizer(f, num_grad_f, x0, H_ones)]
end_time = time.time()
print("2. Оптимальная точка(BFGS с численным градиентом):", result_bfgs2)
print("Значение целевой функции в оптимальной точке:", round(f(result_bfgs2), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")

start_time = time.time()
result_lbfgs1=[round(num, 5) for num in lbfgs_optimizer(f, grad_f, x0, H_ones)]
end_time = time.time()
print("3. Оптимальная точка(L-BFGS с аналитическим градиентом):", result_lbfgs1)
print("Значение целевой функции в оптимальной точке:", round(f(result_lbfgs1), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")

start_time = time.time()
result_lbfgs2=[round(num, 5) for num in lbfgs_optimizer(f, num_grad_f, x0, H_ones)]
end_time = time.time()
print("4. Оптимальная точка(L-BFGS с численным градиентом):", result_lbfgs2)
print("Значение целевой функции в оптимальной точке:", round(f(result_lbfgs2), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")

1. Оптимальная точка(BFGS с аналитическим градиентом): [0.75001, 0.49997, 0.25003]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.00682 с 

2. Оптимальная точка(BFGS с численным градиентом): [0.75001, 0.49997, 0.25003]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.01123 с 

3. Оптимальная точка(L-BFGS с аналитическим градиентом): [0.75, 0.50082, 0.25]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.00787 с 

4. Оптимальная точка(L-BFGS с численным градиентом): [0.75, 0.50082, 0.25]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.00516 с 



Как видно из полученных данных, алгоритмы L-BFGS работают быстрее алгоритмов BFGS (минимум в 4 раза). Это объясняется тем, что алгоритм L-BFGS использует ограниченный объем памяти и вычисляет аппроксимацию матрицы Гессе на основе предыдущих шагов и градиентов.

При аналитическом задании градиента оба алгоритма работают быстрее варианта с численным заданием градиента, т.к. требуется дополнительное время на вычисление приближенного значения градиента.

In [ ]:
#Сравнение алгоритмов при применении аналитического и численного градиента при инициализации Гессиана через вычисленные на основе конечных разностей частные производные целевой функции
start_time = time.time()
result_bfgs1=[round(num, 5) for num in bfgs_optimizer(f, grad_f, x0, H_approx_initial)]
end_time = time.time()
print("Оптимальная точка(BFGS с аналитическим градиентом):", result_bfgs1)
print("Значение целевой функции в оптимальной точке:", round(f(result_bfgs1), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")


start_time = time.time()
result_bfgs2=[round(num, 5) for num in bfgs_optimizer(f, num_grad_f, x0, H_approx_initial)]
end_time = time.time()
print("Оптимальная точка(BFGS с численным градиентом):", result_bfgs2)
print("Значение целевой функции в оптимальной точке:", round(f(result_bfgs2), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")

start_time = time.time()
result_lbfgs1=[round(num, 5) for num in lbfgs_optimizer(f, grad_f,x0, H_approx_initial)]
end_time = time.time()
print("Оптимальная точка(L-BFGS с аналитическим градиентом):", result_lbfgs1)
print("Значение целевой функции в оптимальной точке:", round(f(result_lbfgs1), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")

start_time = time.time()
result_lbfgs2=[round(num, 5) for num in lbfgs_optimizer(f, num_grad_f, x0, H_approx_initial)]
end_time = time.time()
print("Оптимальная точка(L-BFGS с численным градиентом):", result_lbfgs2)
print("Значение целевой функции в оптимальной точке:", round(f(result_lbfgs2), 5))
print("Время выполнения:", round(end_time - start_time, 5), "с","\n")

Оптимальная точка(BFGS с аналитическим градиентом): [0.75, 0.5, 0.25]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.00947 с 

Оптимальная точка(BFGS с численным градиентом): [0.75, 0.5, 0.25]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.01227 с 

Оптимальная точка(L-BFGS с аналитическим градиентом): [0.75053, 0.50075, 0.25037]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.00456 с 

Оптимальная точка(L-BFGS с численным градиентом): [0.75053, 0.50075, 0.25037]
Значение целевой функции в оптимальной точке: -0.375
Время выполнения: 0.00187 с 



При использовании инициализации гессиана через вычисленные на основе конечных разностей частные производные целевой функции алгоритм BFGS и L-BFGS работают быстрее в случае численного задания градиента.

Если же градиент задан аналитически, то лучше использовать инициализацию гессиана единичной матрицей.

### 2. Практическое применение: использование метода оптимизации L-BFGS для решения задачи логистической регрессии

##### Представление набора данных

In [ ]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

In [ ]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [ ]:
df.shape

(569, 31)

In [ ]:
X = df.drop('target', axis=1).values  # Признаки
y = df.target.values  # Целевая переменная (0 - злокачественная опухоль, 1 - доброкачественная опухоль)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##### Реализация логистической регрессии

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def f(theta):
    m = len(y_train)
    h = sigmoid(X_train.dot(theta))
    epsilon = 1e-5

    J = (-1/m) * (y_train.dot(np.log(h + epsilon)) + (1 - y_train).dot(np.log(1 - h + epsilon))) + (lambda_reg / (2 * m)) * np.sum(theta[1:]**2)

    return J

def grad(theta):
    m = len(y_train)
    h = sigmoid(X_train.dot(theta))

    gradient = (1/m) * X_train.T.dot(h - y_train) + (lambda_reg / m) * np.r_[0, theta[1:]]

    return gradient

initial_theta = np.zeros(X_train.shape[1])

lambda_reg = 1

def LBFGS_test_H_ones():

    # Минимизация функции потерь с помощью BFGS bfgs_optimizer(f, grad_f, x0, max_iter=100, tol=1e-6):
    start_time = time.time()
    optimized_theta = lbfgs_optimizer(f, grad, initial_theta, H_ones)
    end_time = time.time()

    y_pred = sigmoid(X_test.dot(optimized_theta))

    threshold = 0.5

    y_pred_binary = (y_pred > threshold).astype(int)
    accuracy = accuracy_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)

    accuracy = round(accuracy, 4)
    precision = round(precision, 4)
    recall = round(recall, 4)
    f1 = round(f1, 4)

    print("LBFGS_H_ones")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", f1)
    print("Time:", end_time-start_time)



def LBFGS_test_H_approx_initial():

    # Минимизация функции потерь с помощью BFGS bfgs_optimizer(f, grad_f, x0, max_iter=100, tol=1e-6):
    # def H_approx_initial(x, f=f, h=1e-6):
    start_time = time.time()
    optimized_theta = lbfgs_optimizer(f, grad, initial_theta, H_approx_initial)
    end_time = time.time()

    y_pred = sigmoid(X_test.dot(optimized_theta))

    threshold = 0.5

    y_pred_binary = (y_pred > threshold).astype(int)
    accuracy = accuracy_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)

    accuracy = round(accuracy, 4)
    precision = round(precision, 4)
    recall = round(recall, 4)
    f1 = round(f1, 4)

    print("LBFGS_H_approx_initial")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", f1)
    print("Time:", end_time-start_time)


##### Сравнение характеристики работы алгоритмов

In [ ]:
LBFGS_test_H_ones()
print("------------")
LBFGS_test_H_approx_initial()

LBFGS_H_ones
Accuracy: 0.9825
Precision: 0.9859
Recall: 0.9859
F1-Score: 0.9859
Time: 0.5061604976654053
------------
LBFGS_H_approx_initial
Accuracy: 0.9035
Precision: 0.9412
Recall: 0.9014
F1-Score: 0.9209
Time: 0.35474061965942383


Алгоритм L-BFGS работает эффективно и дает хорошие результаты для задачи логистической регрессии.

Использование инициализации матрицы Гессе через единичную матрицу приводит к более высокой точности по сравнению с использованием инициализации через вычисленные на основе конечных разностей частные производные целевой функции.

Оба варианта показывают хорошую производительность, и разница во времени выполнения небольшая.